<a href="https://colab.research.google.com/github/ludwigwittgenstein2/Research/blob/master/Name_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Name Entity Recognition using BERT model

##How to use Name Entity Recognition on Custom Dataset using SPACY3? 

1. Dataset 
2. Spacy Library
3. Training

First We require to annotate our custom dataset: 

a. NER Annotator 

https://tecoholic.github.io/ner-annotator/

b. We require our dataset to be in txt file to be uploaded

c. In the NER annotator, apply labels that you desire 

d. Use Export Annotation to export your annotated dataset



# What next? 

a. Once you have your training dataset

b. Use Spacy3




In [6]:
! pip install -U spacy -q

     |████████████████████████████████| 6.4 MB 5.3 MB/s 


In [7]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!python -m spacy info

2022-11-15 05:04:22.378221: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================== Info about spaCy ==============================

spaCy version    3.4.2                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.15                        
Pipelines        en_core_web_sm (3.4.1)        



In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [9]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [10]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 16/16 [00:00<00:00, 1324.27it/s]


In [52]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

In [1]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [11]:
import torch
import spacy

In [13]:
class NerDataset(torch.utils.data.Dataset):
  """
  Custom dataset implementation to get (text,labels) tuples
  Inputs:
   - df : dataframe with columns [tags, sentence]
  """
  
  def __init__(self, df):
    if not isinstance(df, pd.DataFrame):
      raise TypeError('Input should be a dataframe')
    
    if "tags" not in df.columns or "sentence" not in df.columns:
      raise ValueError("Dataframe should contain 'tags' and 'sentence' columns")

     
    
    tags_list = [i.split() for i in df["tags"].values.tolist()]
    texts = df["sentence"].values.tolist()

    self.texts = [tokenizer(text, padding = "max_length", truncation = True, return_tensors = "pt") for text in texts]
    self.labels = [match_tokens_labels(text, tags) for text,tags in zip(self.texts, tags_list)]

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    batch_text = self.texts[idx]
    batch_labels = self.labels[idx]

    return batch_text, torch.LongTensor(batch_labels)

In [14]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2022-11-15 05:05:07.100835: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy


2022-11-15 05:05:14.571847: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-11-15 05:05:15,506] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-11-15 05:05:15,519] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-11-15 05:05:15,524] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-11-15 05:05:15,525] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-11-15 05:05:15,706] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #  

In [16]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

KeyboardInterrupt: ignored

In [17]:
nlp_ner = spacy.load("/content/model-best")

/usr/local/lib/python3.7/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [20]:
doc = nlp_ner('Mr. Breonia Adam , PhD, QC, "	"513 LANDWYCK LN, FLOWER MOUND ,TX 75028') # input sample text
#doc = nlp_ner('Mr. Jasper Bauer , Jr, IQCP, 11096 GINGERWOOD WAY, RANCHO CORDOVA ,CA 95670')

In [21]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

#Compare with Transformer 

In [111]:
# install spacy library
!pip install -U spacy

# Install spacy-transformers
!pip install spacy[transformers]

# Download the traditional spacy english language large model
!python -m spacy download en_core_web_lg 

# Download the spacy transformer (roberta-base) english model
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-11-14 19:43:39.459217: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 15 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
2022-11-14 19:44:28.370444: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 27 kB/s 
✔ Download and installation successful
You can now load the pac

In [112]:
def print_entities(pipeline, text):
    
    # Create a document 
    document = pipeline(text)
    
    # Entity text & label extraction
    for entity in document.ents:
        print(entity.text + '->', entity.label_)
        
        
def visualize_entities(pipeline, text):
    
    # Create a document 
    document = pipeline(text)
        
    # Show entities in pretty manner
    displacy.render(document, jupyter=True, style='ent')

In [113]:

import spacy
from spacy import displacy

# Load English large model
nlp_sm = spacy.load("en_core_web_lg")

In [115]:
short_text = '“Amy Schneider, an engineering manager from Oakland, California, became the first woman and the fourth person on “Jeopardy!” to earn more than $1 million in winnings on Friday’s episode”.'

In [116]:
long_text = 'Good news for consumers, undoubtedly, and good news also for investors. Apple’s recent results, covering the three months to December 31 2016, saw the company’s chief financial officer Luca Maestri announce: ‘We returned nearly $15 billion to investors through share re-purchases and dividends during the quarter.’ The quarterly dividend itself was 57 cents a share, identical to the dividend for the previous three quarters and up on the 52 cents paid for each of the four quarters before that.Business is brisk at Apple. On January 31, Tim Cook, Apple’s chief executive, said of the last three months of 2016: ‘We’re thrilled to report that our holiday quarter results generated Apple’s highest quarterly revenue ever, and broke multiple records along the way. We sold more iPhones than ever before and set all-time revenue records for iPhone, Services, Mac and Apple Watch'

In [128]:
addresss = "Dean. Alex Abadi , Jr, QA, "	"14646 RANCHERO RD, HESPERIA ,CA 92345"

In [117]:
print_entities(nlp_sm, short_text)

Amy Schneider-> PERSON
Oakland-> GPE
California-> GPE
first-> ORDINAL
fourth-> ORDINAL
Jeopardy-> WORK_OF_ART
more than $1 million-> MONEY
Friday-> DATE


In [129]:
print_entities(nlp_sm, addresss)

Dean-> PERSON
Alex Abadi-> PERSON
Jr-> PERSON


In [130]:
visualize_entities(nlp_sm, addresss)

In [118]:
visualize_entities(nlp_sm, short_text)

In [119]:
print_entities(nlp_sm, long_text)

Apple-> ORG
the three months to December 31 2016-> DATE
Luca Maestri-> PERSON
nearly $15 billion-> MONEY
the quarter-> DATE
quarterly-> DATE
57 cents-> MONEY
the previous three quarters-> DATE
the 52 cents-> MONEY
the four quarters-> DATE
Apple-> ORG
January 31-> DATE
Tim Cook-> PERSON
Apple-> ORG
the last three months of 2016-> DATE
our holiday quarter-> DATE
Apple-> ORG
quarterly-> DATE
iPhones-> ORG
iPhone, Services-> ORG
Mac-> ORG
Apple Watch-> ORG


In [120]:
visualize_entities(nlp_sm, short_text)

In [121]:
# Load the spacy transformer (roberta-base) model
roberta_nlp = spacy.load("en_core_web_trf")

In [122]:
print_entities(roberta_nlp, short_text)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Amy Schneider-> PERSON
Oakland-> GPE
California-> GPE
first-> ORDINAL
fourth-> ORDINAL
Jeopardy-> WORK_OF_ART
more than $1 million-> MONEY
Friday-> DATE


In [123]:
visualize_entities(roberta_nlp, short_text)

In [74]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [124]:
print_entities(roberta_nlp, long_text)

Apple-> ORG
the three months to December 31 2016-> DATE
Luca Maestri-> PERSON
nearly $15 billion-> MONEY
the quarter-> DATE
quarterly-> DATE
57 cents-> MONEY
the previous three quarters-> DATE
the 52 cents-> MONEY
the four quarters-> DATE
Apple-> ORG
January 31-> DATE
Tim Cook-> PERSON
Apple-> ORG
the last three months of 2016-> DATE
our holiday quarter-> DATE
Apple-> ORG
quarterly-> DATE
iPhones-> PRODUCT
iPhone-> PRODUCT
Services-> PRODUCT
Mac-> PRODUCT
Apple Watch-> PRODUCT


In [131]:
print_entities(roberta_nlp, addresss)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


QA-> GPE
14646-> CARDINAL
RANCHERO RD-> FAC
HESPERIA-> GPE
CA-> GPE
92345-> CARDINAL


In [126]:
visualize_entities(roberta_nlp, short_text)

In [132]:
visualize_entities(roberta_nlp, addresss)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [125]:
visualize_entities(roberta_nlp, long_text)